# Day 3 task - SQL via Python: NYC School Data Exploration

## 0. Importing libraries

In [41]:
import pandas as pd
import psycopg2


## 1. Connect to the Database

In [47]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

# 1a. Run a Test Query

In [52]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

/var/folders/wq/2ycxrmvj4l97y1mnx5mjmdt80000gn/T/ipykernel_27811/1731849884.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


## 2. School Distribution
- How many schools are there in each borough?

In [53]:
query = """
SELECT borough, COUNT(distinct dbn) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough;
"""
df_result = pd.read_sql(query, conn)
df_result


/var/folders/wq/2ycxrmvj4l97y1mnx5mjmdt80000gn/T/ipykernel_27811/2092121517.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,school_count
0,Bronx,118
1,Brooklyn,121
2,Manhattan,106
3,Queens,80
4,Staten Island,10


## 3. Language Learners
- What is the average % of English Language Learners (ELL) per borough?

for this task a join of the two datasets school_demographics and high_school_directory has to be done as the borough is not available in the school_demographics dataset. Same applies for the next task School supporting special needs
CTE has been created to join the tables

In [54]:
query = """
with data_joined as (
select dire.borough, dire.dbn, demo.ell_num, demo.ell_percent 
from nyc_schools.school_demographics as demo
left join nyc_schools.high_school_directory as dire on demo.dbn = dire.dbn
)

select borough, avg(ell_percent) as avg_ell
from data_joined
group by borough;
"""
df_result = pd.read_sql(query, conn)
df_result

/var/folders/wq/2ycxrmvj4l97y1mnx5mjmdt80000gn/T/ipykernel_27811/2748950647.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,avg_ell
0,None,12.708867
1,Manhattan,7.572500


### Remark: dbn only fits for Manhattan, other dbn of school_demographics are not available in high_school_directory

## 4. School supporting special needs
- find the top3 schools in each borough with the highest percentage of special education students
- cte and window function applied

In [55]:
query = """
 with data_joined as (
select dire.borough, dire.dbn, demo.sped_percent, dire.school_name, demo.schoolyear 
from nyc_schools.school_demographics as demo
left join nyc_schools.high_school_directory as dire on demo.dbn = dire.dbn
)

SELECT borough, dbn, school_name, schoolyear, sped_percent  
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY borough
               ORDER BY sped_percent DESC
           ) AS rn
    FROM data_joined         
) ranked
WHERE rn <= 3
ORDER BY borough, sped_percent DESC;
"""
df_result = pd.read_sql(query, conn)
df_result

/var/folders/wq/2ycxrmvj4l97y1mnx5mjmdt80000gn/T/ipykernel_27811/1179670831.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,dbn,school_name,schoolyear,sped_percent
0,Manhattan,01M450,East Side Community School,20092010,28.8
1,Manhattan,01M450,East Side Community School,20102011,27.7
2,Manhattan,01M450,East Side Community School,20082009,26.7
3,None,None,None,20102011,35.5
4,None,None,None,20102011,35.3
5,None,None,None,20092010,32.6


## 5. Insights
- for joins of the 2 datasets school_demographics and high_school_directory (required for the last 2 questions) only dbn for Manhattan is available in both datasets 
- Staten Island has the lowest number of schools 
- Brooklyn has the highest number of schools
- for the language learners the variety across schools is quite large, overall average is therefore quite small
- school_demographics contains multiple rows for the same school (assume one row for each schoolyear)
